In [1]:
import mne
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import json
from torch import tensor
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import label_binarize

mne.set_log_level('ERROR')

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
import argparse
import os
gpus = [0]
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, gpus))
import numpy as np
import math
import glob
import random
import itertools
import datetime
import time
import datetime
import sys
import scipy.io

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchsummary import summary
import torch.autograd as autograd
from torchvision.models import vgg19

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn.init as init

from torch.utils.data import Dataset
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from sklearn.decomposition import PCA

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
# from common_spatial_pattern import csp

In [2]:
class SeizureSense(nn.Module):
    
    def __init__(self):
        super(SeizureSense,self).__init__()
        
        #Block 1
        #should be taking in an input of 23x512
        #first layer temporal filters
        self.conv1=nn.Conv2d(1,8,(1,128),stride=1,padding=0)
        self.batchnorm1 = nn.BatchNorm2d(8, False)
        
        #spatial layer(depthwise layer)
        self.conv2=nn.Conv2d(8,32,(23,1))
        self.batchnorm2 = nn.BatchNorm2d(32, False)
        self.avgpool1 = nn.AvgPool2d((1,2))
        #apply dropout here in forward
        
        
        #Block 2
        #sepereable convolutional 2d
        self.conv3=nn.Conv2d(32,32,(1,16),stride=1)
        self.batchnorm3 = nn.BatchNorm2d(32,False) #CHECK THIS!
        self.avgpool2 = nn.AvgPool2d((1,16))

        
        #Block 3
        
        self.fc1= nn.Linear(96, 30)
        self.fc2=nn.Linear(30,3)

        #apply dropout here in forward
        self.dropout = nn.Dropout(0.25)
        
    def forward(self,x):
        x=self.conv1(x)
        x =F.elu(x)
        x=self.batchnorm1(x)
        x = self.dropout(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x=self.batchnorm2(x)
        x = self.dropout(x)
        x=self.avgpool1(x)
        
        x=F.elu(self.conv3(x))
        x=self.batchnorm3(x)
        x = self.dropout(x)
        x=self.avgpool2(x)
        
        #Fully connected time
        x=x.view(-1,96)
        x=F.elu(self.fc1(x))
        # FOR TRAINING NO NEED FOR SIGMOUD ON LAST THING BC CROSS ENTRTOPY LOSS SO ADD IT IN INFERENCE DONT FORGET
        x=self.fc2(x)
        return x

In [3]:
model=SeizureSense()
state_dict = torch.load(r"C:\Users\uddha\Downloads\SeizureSenseStateDict (3).pth",map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [27]:
optimal_threshold=.34 #actual=.087
def decode_predictions(predictions):
    agregated_outputs=predictions.mean(dim=1)
    probabilities=F.softmax(agregated_outputs,dim=-1)
    print(probabilities)
    
    if probabilities[0][1]>optimal_threshold:
        return 'preictal'
    else:
        max_values, max_indices = torch.max(probabilities, dim=1)
        if max_indices==0:
            return 'interictal'
        else:
            return 'ictal'
    

In [18]:
def decode_predictions_max(predictions):
    label_names = ['interictal', 'preictal', 'ictal']
    agregated_outputs=predictions.mean(dim=1)
    probabilities=F.softmax(agregated_outputs,dim=-1)
    max_values, max_indices = torch.max(probabilities, dim=1)
    return label_names[max_indices]

    

In [ ]:
#loading data
raw = mne.io.read_raw_edf(r"C:\Users\uddha\Downloads\chb02_19.edf")
raw.load_data()
#Interictal time steps
#2000 seceonn-2001 seceonds
#Preictal 
#2779 seceonds-2780 seceonds
#inseceonds to 3371 seceonds

In [7]:
numpy_array=raw.get_data()
segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
print(segment_tensor.shape)

torch.Size([1, 1, 23, 921600])


In [8]:
def preprocessing(raw):

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)


    freqs = (60,120)
    raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
    
    raw.notch_filter(freqs=freqs, picks=eeg_picks)  # Apply notch filter
    raw.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
    numpy_array=raw.get_data()
    segment_tensor=torch.from_numpy(numpy_array).float().unsqueeze(0).unsqueeze(0)
    return(segment_tensor)

In [9]:
segment_tensor=preprocessing(raw)
i=2779
input=segment_tensor[:,:,:,256*i:256*(i+1)].float()

In [10]:
output=model(input)
output=F.softmax(output,dim=1)
print(output)

tensor([[9.9913e-01, 6.9336e-04, 1.7779e-04]], grad_fn=<SoftmaxBackward0>)


In [11]:
torch.save(segment_tensor,'segment_tensor.pt')

In [20]:
CHB_files={'CHB-MIT/CHB_Database/chb02/chb02_19.edf': [(0, 2769, 'interictal'), (2769, 3369, 'preictal'), (3369, 3378, 'ictal'), (3378, 3600, 'interictal')],}
true_labels=[]
start,end=2778,3388
for i in range(start,end):
    for interval in CHB_files.values():
        for start_interval,end_interval,label in interval:
            if start_interval <= i < end_interval:
                true_labels.append(label)
                break
        else:
            continue
        break
    


In [13]:
class PatchEmbedding(nn.Module):
    def __init__(self, emb_size=40):
        # self.patch_size = patch_size
        super().__init__()

        self.shallownet = nn.Sequential(
            nn.Conv2d(1, 40, (1, 25), (1, 1)),
            nn.Conv2d(40, 40, (22, 1), (1, 1)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.AvgPool2d((1, 75), (1, 15)),  # pooling acts as slicing to obtain 'patch' along the time dimension as in ViT
            nn.Dropout(0.5),
        )

        self.projection = nn.Sequential(
            nn.Conv2d(40, emb_size, (1, 1), stride=(1, 1)),  # transpose, conv could enhance fiting ability slightly
            Rearrange('b e (h) (w) -> b (h w) e'),
        )


    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape
        x = self.shallownet(x)
        x = self.projection(x)
        return x
    
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size, num_heads, dropout):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        self.keys = nn.Linear(emb_size, emb_size)
        self.queries = nn.Linear(emb_size, emb_size)
        self.values = nn.Linear(emb_size, emb_size)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x: Tensor, mask: Tensor = None) -> Tensor:
        queries = rearrange(self.queries(x), "b n (h d) -> b h n d", h=self.num_heads)
        keys = rearrange(self.keys(x), "b n (h d) -> b h n d", h=self.num_heads)
        values = rearrange(self.values(x), "b n (h d) -> b h n d", h=self.num_heads)
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys)  
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)

        scaling = self.emb_size ** (1 / 2)
        att = F.softmax(energy / scaling, dim=-1)
        att = self.att_drop(att)
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x


class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size, expansion, drop_p):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )


class GELU(nn.Module):
    def forward(self, input: Tensor) -> Tensor:
        return input*0.5*1.0+torch.erf(input/math.sqrt(2.0))

class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size,
                 num_heads=10,
                 drop_p=0.5,
                 forward_expansion=4,
                 forward_drop_p=0.5):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, num_heads, drop_p),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))


class TransformerEncoder(nn.Sequential):
    def __init__(self, depth, emb_size):
        super().__init__(*[TransformerEncoderBlock(emb_size) for _ in range(depth)])


class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size, n_classes, d_hidden):
        super().__init__()
        
        # global average pooling
        self.clshead = nn.Sequential(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, n_classes)
        )
        self.fc = nn.Sequential(
            nn.Linear(emb_size, d_hidden),
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(d_hidden, n_classes)
        )

    def forward(self, x):
        #x = x.contiguous().view(x.size(0), -1)
        out = self.fc(x)
        return out



class Conformer(nn.Sequential):
    def __init__(self, emb_size=40, depth=6, n_classes=3, **kwargs):
        super().__init__(

            PatchEmbedding(emb_size),
            TransformerEncoder(depth, emb_size),
            ClassificationHead(emb_size, n_classes, 256)
        )
model=Conformer()

In [28]:
#loop within raw file

model.eval()

segment_tensor=preprocessing(raw)
predictions=[]
if_it_picked_the_max=[]
with torch.no_grad():  # Disable gradient computation for inference

    for i in range(2778,3388):
        input=segment_tensor[:,:,:,256*i:256*(i+1)].float()
        output=model(input)
        output.mean(axis=1)
        



        predicted_label_names=decode_predictions(output)
        predicted_max=decode_predictions_max(output)
        predictions.append(predicted_label_names)
        if_it_picked_the_max.append(predicted_max)
    

tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4310, 0.2927]])
tensor([[0.2763, 0.4

In [26]:
df = pd.DataFrame([predictions, if_it_picked_the_max,true_labels])
csv_file_path = 'label_comparisons.csv'
df.to_csv(csv_file_path, index=False)

In [25]:
count_with_threshold = 0
count_with_max=0

for i in range(len(true_labels)):
    # Assuming predictions[i] is a list with only one element, compare true_labels[i] to predictions[i][0]
    if true_labels[i] == predictions[i]:
        count_with_threshold += 1
    if true_labels[i]==if_it_picked_the_max[i]:
        count_with_max+=1

print('The number of correct predicitons using the pretrained model and threshold',count_with_threshold)
print('The number of correct predictions of pretrained model using torch.max',count_with_max)



The number of correct predicitons using the pretrained model and threshold 591
The number of correct predictions of pretrained model using torch.max 591
